# Baseline

In [100]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_validate, StratifiedGroupKFold, KFold

## Загрузка данных

In [101]:
train_df = pd.read_parquet("train_data.pqt")
test_df = pd.read_parquet("test_data.pqt")

In [102]:

missing_city_when_city_type = train_df[train_df['city_type'].notna() & train_df['city'].isna()]
missing_city_type_when_city = train_df[train_df['city'].notna() & train_df['city_type'].isna()]

missing_city_when_city_type2 = train_df[train_df['index_city_code'].notna() & train_df['city'].isna()]
missing_city_type_when_city2 = train_df[train_df['city'].notna() & train_df['index_city_code'].isna()]


print(f"Количество пропусков в 'city', когда 'city_type' имеет данные: {missing_city_when_city_type.shape[0]}")
print(f"Количество пропусков в 'city_type', когда 'city' имеет данные: {missing_city_type_when_city.shape[0]}")

print(f"Количество пропусков в 'city', когда 'index_city_code' имеет данные: {missing_city_when_city_type2.shape[0]}")
print(f"Количество пропусков в 'index_city_code', когда 'city' имеет данные: {missing_city_type_when_city2.shape[0]}")


Количество пропусков в 'city', когда 'city_type' имеет данные: 0
Количество пропусков в 'city_type', когда 'city' имеет данные: 0
Количество пропусков в 'city', когда 'index_city_code' имеет данные: 32836
Количество пропусков в 'index_city_code', когда 'city' имеет данные: 279716


In [103]:
# # Заполняем пропущенные значения в 'city' значениями из 'index_city_code', если 'city' пустой
# train_df.loc[train_df['city'].isna(), 'city'] = train_df['index_city_code']


In [104]:
# missing_city_when_city_type2 = train_df[train_df['index_city_code'].notna() & train_df['city'].isna()]
# missing_city_type_when_city2 = train_df[train_df['city'].notna() & train_df['index_city_code'].isna()]
#
#
# print(f"Количество пропусков в 'city', когда 'index_city_code' имеет данные: {missing_city_when_city_type2.shape[0]}")
# print(f"Количество пропусков в 'index_city_code', когда 'city' имеет данные: {missing_city_type_when_city2.shape[0]}")

In [105]:
city_counts = train_df['city'].value_counts().to_dict()
#city_counts
train_df["city_other"] = train_df["city"].apply(lambda x: x  if city_counts.get(x, 0) > 10  else "other")
train_df.drop("city", axis=1, inplace=True)
train_df.city_other.value_counts()

city_to_index_mapping = train_df.dropna(subset=['city_other', 'index_city_code']).drop_duplicates('index_city_code').set_index('index_city_code')['city_other'].to_dict()


for idx, row in train_df.iterrows():
    if pd.isna(row['city_other']):
        index_city_code = row['index_city_code']

        if index_city_code in city_to_index_mapping:
            train_df.at[idx, 'city_other'] = city_to_index_mapping[index_city_code]

        elif not pd.isna(index_city_code):
            train_df.at[idx, 'city_other'] = index_city_code

train_df["city"] = train_df["city_other"]
train_df.drop("city_other", axis=1, inplace=True)

In [106]:
train_df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city_type,index_city_code,ogrn_days_end_month,...,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster,city
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_type_0,index_city_code_39,-0.488553,...,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other},city_23
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_type_0,index_city_code_39,-0.488553,...,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other},city_23
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_type_0,index_city_code_39,-0.488553,...,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other},city_23
3,1,month_1,-0.081586,-0.091860,-0.114040,-0.080890,channel_code_2,city_type_0,None,0.324343,...,0.407762,0.369318,0.567093,0.785465,-0.184002,0.253523,0.462452,{other},{other},city_14
4,1,month_2,-0.094962,-0.100504,-0.119302,-0.094307,channel_code_2,city_type_0,None,0.324343,...,0.430750,0.067275,0.559928,0.696576,-0.183854,0.255545,0.495419,{other},{other},city_14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,199998,month_2,-0.156775,-0.204960,-0.125987,-0.156311,channel_code_9,city_type_0,None,0.324343,...,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{},city_14
599996,199998,month_3,-0.156776,-0.204960,-0.125995,-0.156312,channel_code_9,city_type_0,None,0.324343,...,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,{α},{},city_14
599997,199999,month_1,-0.156712,-0.204913,-0.125831,-0.156248,channel_code_14,city_type_0,index_city_code_195,1.717879,...,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{},city_1876
599998,199999,month_2,-0.156712,-0.204913,-0.125831,-0.156248,channel_code_14,city_type_0,index_city_code_195,1.717879,...,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{},city_1876


In [107]:
train_df.head(30)

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city_type,index_city_code,ogrn_days_end_month,...,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster,city
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_type_0,index_city_code_39,-0.488553,...,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other},city_23
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_type_0,index_city_code_39,-0.488553,...,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other},city_23
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_type_0,index_city_code_39,-0.488553,...,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other},city_23
3,1,month_1,-0.081586,-0.091860,-0.114040,-0.080890,channel_code_2,city_type_0,None,0.324343,...,0.407762,0.369318,0.567093,0.785465,-0.184002,0.253523,0.462452,{other},{other},city_14
4,1,month_2,-0.094962,-0.100504,-0.119302,-0.094307,channel_code_2,city_type_0,None,0.324343,...,0.430750,0.067275,0.559928,0.696576,-0.183854,0.255545,0.495419,{other},{other},city_14
5,1,month_3,-0.090605,-0.114275,-0.114119,-0.089937,channel_code_2,city_type_0,None,0.324343,...,0.488221,0.043221,0.560788,0.707687,-0.167905,0.259011,0.605309,{other},{other},city_14
6,2,month_1,-0.154685,-0.186795,-0.122805,-0.154215,channel_code_12,city_type_306,None,-0.256297,...,0.384773,-0.165588,0.546889,0.407687,-0.178674,0.252657,0.440474,{α},{α},city_613
7,2,month_2,-0.152784,-0.193686,-0.122805,-0.152308,channel_code_12,city_type_306,None,-0.256297,...,0.384773,-0.165588,0.546889,0.407687,-0.177854,0.252657,0.440474,{α},{α},city_613
8,2,month_3,-0.148737,-0.187003,-0.112416,-0.148249,channel_code_12,city_type_306,None,-0.256297,...,0.384773,-0.165588,0.546889,0.407687,-0.176302,0.252368,0.429485,{α},{α},city_613
9,3,month_1,-0.156643,-0.204861,-0.125660,-0.156179,channel_code_14,city_type_0,index_city_code_46,-1.185321,...,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,{α},{α},city_21


In [108]:

train_df['month_number'] = train_df.groupby('id').cumcount() + 1


train_df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city_type,index_city_code,ogrn_days_end_month,...,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster,city,month_number
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_type_0,index_city_code_39,-0.488553,...,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other},city_23,1
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_type_0,index_city_code_39,-0.488553,...,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other},city_23,2
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_type_0,index_city_code_39,-0.488553,...,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other},city_23,3
3,1,month_1,-0.081586,-0.091860,-0.114040,-0.080890,channel_code_2,city_type_0,None,0.324343,...,0.369318,0.567093,0.785465,-0.184002,0.253523,0.462452,{other},{other},city_14,1
4,1,month_2,-0.094962,-0.100504,-0.119302,-0.094307,channel_code_2,city_type_0,None,0.324343,...,0.067275,0.559928,0.696576,-0.183854,0.255545,0.495419,{other},{other},city_14,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,199998,month_2,-0.156775,-0.204960,-0.125987,-0.156311,channel_code_9,city_type_0,None,0.324343,...,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{},city_14,2
599996,199998,month_3,-0.156776,-0.204960,-0.125995,-0.156312,channel_code_9,city_type_0,None,0.324343,...,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,{α},{},city_14,3
599997,199999,month_1,-0.156712,-0.204913,-0.125831,-0.156248,channel_code_14,city_type_0,index_city_code_195,1.717879,...,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{},city_1876,1
599998,199999,month_2,-0.156712,-0.204913,-0.125831,-0.156248,channel_code_14,city_type_0,index_city_code_195,1.717879,...,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{},city_1876,2


In [109]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 94 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       600000 non-null  int64  
 1   date                     600000 non-null  object 
 2   balance_amt_avg          534423 non-null  float64
 3   balance_amt_max          534423 non-null  float64
 4   balance_amt_min          534423 non-null  float64
 5   balance_amt_day_avg      534423 non-null  float64
 6   channel_code             557150 non-null  object 
 7   city_type                521656 non-null  object 
 8   index_city_code          274776 non-null  object 
 9   ogrn_days_end_month      568527 non-null  float64
 10  ogrn_days_end_quarter    568527 non-null  float64
 11  ogrn_month               568527 non-null  object 
 12  ogrn_year                568527 non-null  object 
 13  ft_registration_date     568640 non-null  float64
 14  max_

In [110]:
pivot_columns = [col for col in train_df.columns if col not in ['id', 'date', 'month_number']]

df_pivoted = train_df.pivot_table(index='id', columns='month_number', values=pivot_columns, aggfunc='first')

# Теперь, когда мы сформировали сводную таблицу, преобразуем MultiIndex столбцы в единый уровень
df_pivoted.columns = [f'{col[0]}_{col[1]}month' for col in df_pivoted.columns]

# Переименовываем индекс обратно в столбец 'id', если он стал индексом
df_pivoted.reset_index(inplace=True)

df_pivoted.head()



,id,balance_amt_avg_1month,balance_amt_avg_2month,balance_amt_avg_3month,balance_amt_day_avg_1month,balance_amt_day_avg_2month,balance_amt_day_avg_3month,balance_amt_max_1month,balance_amt_max_2month,balance_amt_max_3month,...,sum_deb_h_oper_3m_3month,sum_of_paym_1y_1month,sum_of_paym_1y_2month,sum_of_paym_1y_3month,sum_of_paym_2m_1month,sum_of_paym_2m_2month,sum_of_paym_2m_3month,sum_of_paym_6m_1month,sum_of_paym_6m_2month,sum_of_paym_6m_3month
0,0,0.744845,1.049605,0.692653,0.748101,1.053805,0.695747,0.705492,0.831916,0.740253,...,0.877050,0.511490,0.486425,0.480547,0.942275,0.645704,0.403604,0.536013,0.536378,0.613167
1,1,-0.081586,-0.094962,-0.090605,-0.080890,-0.094307,-0.089937,-0.091860,-0.100504,-0.114275,...,0.043221,0.052041,0.033554,0.039472,0.014051,-0.057593,-0.092059,0.043800,0.035027,0.025233
2,2,-0.154685,-0.152784,-0.148737,-0.154215,-0.152308,-0.148249,-0.186795,-0.193686,-0.187003,...,-0.165588,-0.291924,-0.290712,-0.288318,-0.255837,-0.267913,-0.255946,-0.287121,-0.284955,-0.280676
3,3,-0.156643,-0.156666,-0.156522,-0.156179,-0.156202,-0.156058,-0.204861,-0.204790,-0.204718,...,-0.165588,-0.242793,-0.262878,-0.273303,-0.273969,-0.273969,-0.273969,-0.268832,-0.294398,-0.294447
4,4,-0.138847,-0.129233,-0.141798,-0.138328,-0.128684,-0.141289,-0.182486,-0.162965,-0.170262,...,-0.078297,-0.124641,-0.121939,-0.128903,-0.103807,-0.134192,-0.166740,-0.130025,-0.134049,-0.142831


In [111]:
df_pivoted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 274 entries, id to sum_of_paym_6m_3month
dtypes: float64(243), int64(1), object(30)
memory usage: 418.1+ MB


In [112]:
float_cols = df_pivoted.select_dtypes(include=['float64', 'float32'])

std_devs = float_cols.std()
srted = std_devs.sort_values()

srted.head(20)


cnt_cred_f_oper_3m_1month    0.000886
cnt_cred_e_oper_3m_1month    0.001131
cnt_cred_f_oper_1m_1month    0.001246
cnt_deb_f_oper_3m_1month     0.001289
cnt_cred_e_oper_1m_1month    0.001361
cnt_cred_f_oper_1m_2month    0.001488
cnt_deb_f_oper_1m_1month     0.001514
cnt_cred_f_oper_3m_2month    0.001546
cnt_cred_e_oper_1m_2month    0.001563
cnt_b_oper_3m_3month         0.001655
cnt_cred_e_oper_3m_2month    0.001695
cnt_deb_f_oper_1m_2month     0.001834
cnt_deb_f_oper_3m_2month     0.001894
cnt_deb_e_oper_1m_1month     0.002139
cnt_deb_e_oper_3m_1month     0.002175
cnt_b_oper_3m_2month         0.002196
cnt_deb_e_oper_1m_2month     0.002359
cnt_cred_f_oper_3m_3month    0.002564
cnt_deb_f_oper_3m_3month     0.002612
cnt_deb_e_oper_3m_2month     0.002639
dtype: float64

In [113]:
float_cols = df_pivoted.select_dtypes(include=['float64', 'float32'])

means = float_cols.mean()

std_devs = float_cols.std()

cv_percent = (std_devs / abs(means)) * 100

sorted_cv_percent = cv_percent.sort_values()

print(sorted_cv_percent.iloc[0:50])
print(sorted_cv_percent.iloc[50:20])


cnt_cred_e_oper_3m_1month    0.130930
cnt_cred_e_oper_1m_1month    0.155773
cnt_cred_e_oper_1m_2month    0.178981
cnt_cred_e_oper_3m_2month    0.196164
cnt_deb_f_oper_1m_1month     0.196272
cnt_deb_f_oper_1m_2month     0.237819
cnt_deb_e_oper_1m_1month     0.239462
cnt_deb_e_oper_1m_2month     0.264140
cnt_cred_e_oper_3m_3month    0.307015
cnt_cred_e_oper_1m_3month    0.310253
cnt_b_oper_3m_3month         0.319841
cnt_deb_e_oper_1m_3month     0.345459
cnt_deb_f_oper_1m_3month     0.348912
cnt_cred_f_oper_3m_1month    0.356484
cnt_c_oper_3m_2month         0.358865
cnt_c_oper_3m_1month         0.359786
cnt_c_oper_3m_3month         0.362152
cnt_b_oper_3m_2month         0.424392
cnt_deb_f_oper_3m_1month     0.450445
cnt_deb_e_oper_3m_1month     0.510480
cnt_deb_d_oper_1m_2month     0.588607
cnt_c_oper_1m_1month         0.593846
cnt_b_oper_3m_1month         0.614297
cnt_deb_e_oper_3m_2month     0.619415
cnt_cred_f_oper_3m_2month    0.621906
cnt_deb_d_oper_3m_3month     0.624855
cnt_deb_d_op

In [114]:
# float_columns = df_pivoted.select_dtypes(include=['float64']).columns
# df_pivoted[float_columns] = df_pivoted[float_columns].astype('float32')

In [115]:

X = df_pivoted.drop([
"cnt_a_oper_1m_1month", "cnt_a_oper_1m_2month", "cnt_a_oper_1m_3month",
"cnt_b_oper_1m_1month", "cnt_b_oper_1m_2month", "cnt_b_oper_1m_3month",
"cnt_c_oper_1m_1month", "cnt_c_oper_1m_2month", "cnt_c_oper_1m_3month",
"cnt_deb_d_oper_1m_1month", "cnt_deb_d_oper_1m_2month", "cnt_deb_d_oper_1m_3month",
"cnt_cred_d_oper_1m_1month", "cnt_cred_d_oper_1m_2month", "cnt_cred_d_oper_1m_3month",
"cnt_deb_e_oper_1m_1month", "cnt_deb_e_oper_1m_2month", "cnt_deb_e_oper_1m_3month",
"cnt_cred_e_oper_1m_1month", "cnt_cred_e_oper_1m_2month", "cnt_cred_e_oper_1m_3month",
"cnt_deb_f_oper_1m_1month", "cnt_deb_f_oper_1m_2month", "cnt_deb_f_oper_1m_3month",
"cnt_cred_f_oper_1m_1month", "cnt_cred_f_oper_1m_2month", "cnt_cred_f_oper_1m_3month",
"cnt_deb_g_oper_1m_1month", "cnt_deb_g_oper_1m_2month", "cnt_deb_g_oper_1m_3month",
"cnt_cred_g_oper_1m_1month", "cnt_cred_g_oper_1m_2month", "cnt_cred_g_oper_1m_3month",
"cnt_deb_h_oper_1m_1month", "cnt_deb_h_oper_1m_2month", "cnt_deb_h_oper_1m_3month",
"cnt_cred_h_oper_1m_1month", "cnt_cred_h_oper_1m_2month", "cnt_cred_h_oper_1m_3month",
"cnt_a_oper_3m_1month", "cnt_a_oper_3m_2month", "cnt_a_oper_3m_3month",
"cnt_b_oper_3m_1month", "cnt_b_oper_3m_2month", "cnt_b_oper_3m_3month",
"cnt_c_oper_3m_1month", "cnt_c_oper_3m_2month", "cnt_c_oper_3m_3month",
"cnt_deb_d_oper_3m_1month", "cnt_deb_d_oper_3m_2month", "cnt_deb_d_oper_3m_3month",
"cnt_cred_d_oper_3m_1month", "cnt_cred_d_oper_3m_2month", "cnt_cred_d_oper_3m_3month",
"cnt_deb_e_oper_3m_1month", "cnt_deb_e_oper_3m_2month", "cnt_deb_e_oper_3m_3month",
"cnt_cred_e_oper_3m_1month", "cnt_cred_e_oper_3m_2month", "cnt_cred_e_oper_3m_3month",
"cnt_deb_f_oper_3m_1month", "cnt_deb_f_oper_3m_2month", "cnt_deb_f_oper_3m_3month",
"cnt_cred_f_oper_3m_1month", "cnt_cred_f_oper_3m_2month", "cnt_cred_f_oper_3m_3month",
"cnt_deb_g_oper_3m_1month", "cnt_deb_g_oper_3m_2month", "cnt_deb_g_oper_3m_3month",
"cnt_cred_g_oper_3m_1month", "cnt_cred_g_oper_3m_2month", "cnt_cred_g_oper_3m_3month",
"cnt_deb_h_oper_3m_1month", "cnt_deb_h_oper_3m_2month", "cnt_deb_h_oper_3m_3month",
"cnt_cred_h_oper_3m_1month", "cnt_cred_h_oper_3m_2month", "cnt_cred_h_oper_3m_3month",

"city_type_1month",
"city_type_2month",
"city_type_3month",
"index_city_code_1month",
"index_city_code_2month",
"index_city_code_3month",

"start_cluster_3month",
"id","end_cluster_3month", "end_cluster_2month", "end_cluster_1month"
], axis=1)
y = df_pivoted["end_cluster_3month"]

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

В качестве базовой модели возьмем LGBM обучим на всех признаках

In [116]:
params_cat = {
    "iterations":10000,
    "depth":7,
    "learning_rate":0.02,
    "eval_metric":'AUC',
    "random_seed":1,
    "bootstrap_type":'Bayesian',
    "bagging_temperature":1,
    "od_type":'Iter',
    "od_wait":300,
    "task_type":'GPU'
}

In [117]:
from sklearn.model_selection import StratifiedKFold, LeavePOut, RepeatedStratifiedKFold
from catboost import  CatBoostClassifier


cat_model = CatBoostClassifier(**params_cat)


# cv = KFold(n_splits=7)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=12)

# cv = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
auc_scores = []  # Список для хранения показателей AUC для каждого разбиения


In [118]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [119]:
cluster_weights = pd.read_excel("cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [120]:
# weights_dict

In [121]:
cat_features = [col for col in X.columns if X[col].dtype.name == 'category' or X[col].dtype.name == 'object']
for col in cat_features:
    X[col] = X[col].astype('category')
    #test_df[col] = test_df[col].astype('category')



In [122]:
y = y.astype('category')
# y = y.cat.add_categories('Missing').fillna('Missing')

In [123]:
cat_features = [col for col in X.columns if X[col].dtype.name == 'category' or X[col].dtype.name == 'object']
for col in cat_features:
    X[col] = X[col].cat.add_categories('Missing').fillna('Missing')
cat_features

['channel_code_1month',
 'channel_code_2month',
 'channel_code_3month',
 'city_1month',
 'city_2month',
 'city_3month',
 'ogrn_month_1month',
 'ogrn_month_2month',
 'ogrn_month_3month',
 'ogrn_year_1month',
 'ogrn_year_2month',
 'ogrn_year_3month',
 'okved_1month',
 'okved_2month',
 'okved_3month',
 'segment_1month',
 'segment_2month',
 'segment_3month',
 'start_cluster_1month',
 'start_cluster_2month']

In [124]:
cat_features = [col for col in X.columns if X[col].dtype.name == 'category' or X[col].dtype.name == 'object']
cat_features

['channel_code_1month',
 'channel_code_2month',
 'channel_code_3month',
 'city_1month',
 'city_2month',
 'city_3month',
 'ogrn_month_1month',
 'ogrn_month_2month',
 'ogrn_month_3month',
 'ogrn_year_1month',
 'ogrn_year_2month',
 'ogrn_year_3month',
 'okved_1month',
 'okved_2month',
 'okved_3month',
 'segment_1month',
 'segment_2month',
 'segment_3month',
 'start_cluster_1month',
 'start_cluster_2month']

In [125]:
X

,balance_amt_avg_1month,balance_amt_avg_2month,balance_amt_avg_3month,balance_amt_day_avg_1month,balance_amt_day_avg_2month,balance_amt_day_avg_3month,balance_amt_max_1month,balance_amt_max_2month,balance_amt_max_3month,balance_amt_min_1month,...,sum_deb_h_oper_3m_3month,sum_of_paym_1y_1month,sum_of_paym_1y_2month,sum_of_paym_1y_3month,sum_of_paym_2m_1month,sum_of_paym_2m_2month,sum_of_paym_2m_3month,sum_of_paym_6m_1month,sum_of_paym_6m_2month,sum_of_paym_6m_3month
0,0.744845,1.049605,0.692653,0.748101,1.053805,0.695747,0.705492,0.831916,0.740253,1.287207,...,0.877050,0.511490,0.486425,0.480547,0.942275,0.645704,0.403604,0.536013,0.536378,0.613167
1,-0.081586,-0.094962,-0.090605,-0.080890,-0.094307,-0.089937,-0.091860,-0.100504,-0.114275,-0.114040,...,0.043221,0.052041,0.033554,0.039472,0.014051,-0.057593,-0.092059,0.043800,0.035027,0.025233
2,-0.154685,-0.152784,-0.148737,-0.154215,-0.152308,-0.148249,-0.186795,-0.193686,-0.187003,-0.122805,...,-0.165588,-0.291924,-0.290712,-0.288318,-0.255837,-0.267913,-0.255946,-0.287121,-0.284955,-0.280676
3,-0.156643,-0.156666,-0.156522,-0.156179,-0.156202,-0.156058,-0.204861,-0.204790,-0.204718,-0.125660,...,-0.165588,-0.242793,-0.262878,-0.273303,-0.273969,-0.273969,-0.273969,-0.268832,-0.294398,-0.294447
4,-0.138847,-0.129233,-0.141798,-0.138328,-0.128684,-0.141289,-0.182486,-0.162965,-0.170262,-0.125630,...,-0.078297,-0.124641,-0.121939,-0.128903,-0.103807,-0.134192,-0.166740,-0.130025,-0.134049,-0.142831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,-0.067129,-0.004851,-0.039281,-0.066388,-0.003917,-0.038454,-0.108082,-0.053838,-0.053694,-0.031401,...,-0.078382,-0.295238,-0.290994,-0.283410,-0.269524,-0.248313,-0.214854,-0.293044,-0.285459,-0.271905
199996,0.543173,0.413435,0.293117,0.545804,0.415664,0.294974,0.382148,0.281704,0.189316,1.484547,...,-0.153030,-0.125855,-0.126250,-0.126344,-0.221403,-0.222278,-0.225526,-0.136274,-0.178142,-0.194693
199997,-0.082159,-0.111169,0.032941,-0.081465,-0.110564,0.033992,-0.028263,-0.041417,0.140726,-0.120011,...,0.680322,1.732060,1.662368,1.694414,0.661477,0.127654,0.751383,1.658004,1.251180,1.126610
199998,-0.156775,-0.156775,-0.156776,-0.156311,-0.156311,-0.156312,-0.204960,-0.204960,-0.204960,-0.125987,...,-0.165588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
y_val.info()

<class 'pandas.core.series.Series'>
Index: 40000 entries, 119737 to 6584
Series name: end_cluster_3month
Non-Null Count  Dtype 
--------------  ----- 
40000 non-null  object
dtypes: object(1)
memory usage: 625.0+ KB


In [127]:
i=0
for train_idx, val_idx in cv.split(X, y):
    print(f"\n\ni: {i} ")
    x_train, x_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    cat_model.fit(
        x_train, y_train,
        eval_set=(x_val, y_val),
        cat_features=cat_features,
        use_best_model=True,
        verbose=True
    )
    i+=1
    y_pred_proba = cat_model.predict_proba(x_val)
    cat_model.save_model(f'my_meth6_d7/model_{i}.cbm')
    #i+=1
    auc_score = weighted_roc_auc(y_val, y_pred_proba, cat_model.classes_, weights_dict)
    auc_scores.append(auc_score)


mean_auc = np.mean(auc_scores)
std_auc = np.std(auc_scores)
print(f"Средний взвешенный ROC AUC: {mean_auc:.3f} ± {std_auc:.3f}")




i: 0 


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8113485	best: 0.8113485 (0)	total: 385ms	remaining: 1h 4m 13s
1:	total: 637ms	remaining: 53m 6s
2:	total: 695ms	remaining: 38m 36s
3:	total: 757ms	remaining: 31m 30s
4:	total: 815ms	remaining: 27m 9s
5:	test: 0.8307237	best: 0.8307237 (5)	total: 879ms	remaining: 24m 23s
6:	total: 939ms	remaining: 22m 19s
7:	total: 995ms	remaining: 20m 43s
8:	total: 1.05s	remaining: 19m 30s
9:	total: 1.11s	remaining: 18m 27s
10:	test: 0.8330057	best: 0.8333481 (9)	total: 1.16s	remaining: 17m 35s
11:	total: 1.22s	remaining: 16m 55s
12:	total: 1.28s	remaining: 16m 21s
13:	total: 1.34s	remaining: 15m 54s
14:	total: 1.4s	remaining: 15m 29s
15:	test: 0.8371351	best: 0.8371351 (15)	total: 1.46s	remaining: 15m 8s
16:	total: 1.51s	remaining: 14m 48s
17:	total: 1.57s	remaining: 14m 30s
18:	total: 1.63s	remaining: 14m 15s
19:	total: 1.69s	remaining: 14m 3s
20:	test: 0.8393252	best: 0.8393252 (20)	total: 1.75s	remaining: 13m 51s
21:	total: 1.8s	remaining: 13m 38s
22:	total: 1.86s	remaining: 13m 28s
23:	

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8177992	best: 0.8177992 (0)	total: 185ms	remaining: 30m 54s
1:	total: 305ms	remaining: 25m 24s
2:	total: 431ms	remaining: 23m 56s
3:	total: 490ms	remaining: 20m 24s
4:	total: 541ms	remaining: 18m
5:	test: 0.8251568	best: 0.8251568 (5)	total: 596ms	remaining: 16m 32s
6:	total: 651ms	remaining: 15m 29s
7:	total: 702ms	remaining: 14m 37s
8:	total: 757ms	remaining: 14m
9:	total: 809ms	remaining: 13m 27s
10:	test: 0.8275304	best: 0.8288726 (8)	total: 866ms	remaining: 13m 6s
11:	total: 922ms	remaining: 12m 47s
12:	total: 974ms	remaining: 12m 28s
13:	total: 1.02s	remaining: 12m 10s
14:	total: 1.08s	remaining: 11m 58s
15:	test: 0.8314414	best: 0.8314414 (15)	total: 1.13s	remaining: 11m 47s
16:	total: 1.19s	remaining: 11m 38s
17:	total: 1.24s	remaining: 11m 28s
18:	total: 1.3s	remaining: 11m 23s
19:	total: 1.35s	remaining: 11m 14s
20:	test: 0.8351655	best: 0.8351728 (19)	total: 1.41s	remaining: 11m 11s
21:	total: 1.47s	remaining: 11m 5s
22:	total: 1.52s	remaining: 10m 59s
23:	total: 

...

0:	test: 0.8051900	best: 0.8051900 (0)	total: 93.6ms	remaining: 15m 35s
1:	total: 155ms	remaining: 12m 52s
2:	total: 273ms	remaining: 15m 11s
3:	total: 329ms	remaining: 13m 42s
4:	total: 380ms	remaining: 12m 40s
5:	test: 0.8113752	best: 0.8113752 (5)	total: 433ms	remaining: 12m 1s
6:	total: 485ms	remaining: 11m 31s
7:	total: 542ms	remaining: 11m 16s
8:	total: 594ms	remaining: 10m 59s
9:	total: 649ms	remaining: 10m 48s
10:	test: 0.8308650	best: 0.8308650 (10)	total: 700ms	remaining: 10m 35s
11:	total: 760ms	remaining: 10m 32s
12:	total: 812ms	remaining: 10m 24s
13:	total: 869ms	remaining: 10m 19s
14:	total: 922ms	remaining: 10m 13s
15:	test: 0.8385977	best: 0.8385977 (15)	total: 977ms	remaining: 10m 9s
16:	total: 1.03s	remaining: 10m 4s
17:	total: 1.08s	remaining: 10m
18:	total: 1.14s	remaining: 9m 56s
19:	total: 1.2s	remaining: 9m 57s
20:	test: 0.8430594	best: 0.8430594 (20)	total: 1.25s	remaining: 9m 55s
21:	total: 1.31s	remaining: 9m 53s
22:	total: 1.36s	remaining: 9m 51s
23:	total: 

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8153042	best: 0.8153042 (0)	total: 93.3ms	remaining: 15m 33s
1:	total: 177ms	remaining: 14m 45s
2:	total: 298ms	remaining: 16m 32s
3:	total: 352ms	remaining: 14m 39s
4:	total: 405ms	remaining: 13m 30s
5:	test: 0.8243613	best: 0.8243613 (5)	total: 457ms	remaining: 12m 41s
6:	total: 510ms	remaining: 12m 7s
7:	total: 563ms	remaining: 11m 43s
8:	total: 620ms	remaining: 11m 28s
9:	total: 675ms	remaining: 11m 13s
10:	test: 0.8285053	best: 0.8285053 (10)	total: 727ms	remaining: 11m
11:	total: 785ms	remaining: 10m 53s
12:	total: 838ms	remaining: 10m 43s
13:	total: 890ms	remaining: 10m 35s
14:	total: 940ms	remaining: 10m 26s
15:	test: 0.8335777	best: 0.8335777 (15)	total: 992ms	remaining: 10m 19s
16:	total: 1.04s	remaining: 10m 13s
17:	total: 1.1s	remaining: 10m 10s
18:	total: 1.15s	remaining: 10m 6s
19:	total: 1.21s	remaining: 10m 2s
20:	test: 0.8372774	best: 0.8375001 (18)	total: 1.26s	remaining: 9m 59s
21:	total: 1.31s	remaining: 9m 56s
22:	total: 1.37s	remaining: 9m 55s
23:	total

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8159400	best: 0.8159400 (0)	total: 100ms	remaining: 16m 40s
1:	total: 216ms	remaining: 18m
2:	total: 361ms	remaining: 20m 4s
3:	total: 423ms	remaining: 17m 36s
4:	total: 476ms	remaining: 15m 52s
5:	test: 0.8316255	best: 0.8316255 (5)	total: 535ms	remaining: 14m 50s
6:	total: 589ms	remaining: 14m
7:	total: 642ms	remaining: 13m 21s
8:	total: 692ms	remaining: 12m 48s
9:	total: 748ms	remaining: 12m 27s
10:	test: 0.8368719	best: 0.8368719 (10)	total: 800ms	remaining: 12m 6s
11:	total: 853ms	remaining: 11m 49s
12:	total: 907ms	remaining: 11m 36s
13:	total: 958ms	remaining: 11m 23s
14:	total: 1.01s	remaining: 11m 12s
15:	test: 0.8388383	best: 0.8388383 (15)	total: 1.06s	remaining: 11m 1s
16:	total: 1.11s	remaining: 10m 54s
17:	total: 1.17s	remaining: 10m 46s
18:	total: 1.22s	remaining: 10m 39s
19:	total: 1.27s	remaining: 10m 35s
20:	test: 0.8373516	best: 0.8388383 (15)	total: 1.33s	remaining: 10m 31s
21:	total: 1.39s	remaining: 10m 28s
22:	total: 1.44s	remaining: 10m 24s
23:	total: